In [5]:
#from bs4 import BeautifulSoup
#import requests
#import re
import json

#import gensim
#from gensim.parsing.preprocessing import remove_stopwords
#import nltk
#from nltk import tokenize
#print(nltk.__version__)#3.5
#import regex
#print(regex.__version__)#2.5.83

import spacy
#from spacy.matcher import PhraseMatcher
#from spacy.tokens import Span
#from spacy.language import Language
#from spacy.lang.en import English
from spacy.matcher import Matcher
#from spacy.training import Example
#import random

In [46]:
"""
Function to get links

"""

#URLs
main_url= 'https://www.smithsonianmag.com'
paleo_url= '/tag/paleontology/?page='

#Get links from sections
def getLinksfromPage(url):
    r= requests.get(url)
    soup= BeautifulSoup(r.text, 'html.parser')
    
    #Get grid with title and link
    grids= soup.body.find('div', {'class': 'container'}).find('div', {'class': 'main-wrapper'}).main.div.find_all('section', class_='article-teaser')
    #Links
    links= []
    for grid in grids:
        links.append(
            grid.a['href']
        )
    

    return links

In [ ]:
"""
Function to get article content

"""

r2= '/smart-news/you-can-buy-t-rex-skeleton-hefty-price-180975847/'

def getArticlefromLink(art_link):
    r= requests.get(main_url+art_link)
    soup= BeautifulSoup(r.text, 'html.parser')
    #Main
    main= soup.find('div', class_='main-wrapper').main
    try:
        #Header
        title= main.header.h1.text
        try:
            author= main.header.div.div.find('div', class_='by-line').a.text.strip()
        except:
            author= main.header.div.div.find('div', class_='by-line').span.text.strip()
        date= main.header.div.div.find('div', class_='edition').time.text
        #Body
        body= main.find('div', class_='article-body pagination-first').find_all('p')
        body2= ''
        for par in body:
            body2= body2+par.text+' '
        body2= body2.strip()
    except:
        #Header
        try:
            title= main.div.find('div', class_='main-content').h2.text
        except:
            return None
        try:
            author= main.div.find('div', class_='post-info').find('span', class_='byline').a.text
        except:
            author= main.div.find('div', class_='post-info').find('span', class_='byline').span.text
        date= main.div.find('div', class_='post-info').find('span', class_='date').text
        #Body
        body= main.find('div', class_='main-content').find_all('p')
        body2= ''
        for par in body[:-1]:
            body2= body2+par.text+' '    
        body2= body2.strip()
    
    #Remove signs
    body2= re.sub(r'[\/#!$%\^&\*;:{}=\[\]_`~()“”…—\"\"‘’\'\'>–]', '', body2)
    body2= re.sub(r'[-]', ' ', body2)
    
    doc= {
        'title': title,
        'author': author,
        'date': date,
        'doc': body2
         }

    return(doc)

getArticlefromLink(r2)

In [47]:
"""
Get links and articles content

"""

all_arts= []
for page_i in range(1,44):
    #Join main url and page number
    page_url= main_url+paleo_url+str(page_i)
    #Get links from page i
    all_links= getLinksfromPage(page_url)
    
    #Get articles from page i
    for link_i in all_links:
        all_arts.append(getArticlefromLink(link_i))

#423 total articles
print(len(all_arts))

423


In [50]:
"""
Save json

"""

all_arts= [art for art in all_arts if art]
#397 articles
print(len(all_arts))

#Save json
paleo= {'paleontology': all_arts}
with open('paleo.json', 'w') as fp:
    json.dump(paleo, fp)

397


# 0.5 Open jsons

In [2]:
"""
Open jsons

"""

#Open jsons
with open('paleo.json') as input_file:
    json1 = json.load(input_file)
data1= json1['paleontology']
print(len(data1))

with open('paleo2.json') as input_file:
    json2 = json.load(input_file)
data2= json2['paleontology']
print(len(data2))

#Join jsons
data= data1+data2 #1450 arts

397
1053


# 1. Extinctions/Catastrophes mentions
### Spacy trained model

In [10]:
"""
Spacy trained model: Catastrophes

"""

TEXTS= [
        'Scientists have long believed—at least before humanity became a force for extinction that there were just two ways to wipe out life on Earth: an asteroid strike or massive volcanic eruptions.',
        'What the end-Devonian lacked was a cause. There was no evidence for volcanism or a giant impact, but one alluring clue was seen in the rapid formation.',
        'The Cretaceous-Paleogene extinction event is the most recent mass extinction and the only one definitively connected to a major asteroid impact.',
        'The massive impact which left a crater more than 120 miles wide—flung huge volumes of dust, debris, and sulfur into the atmosphere, bringing on severe global cooling.',
        'According to the Shiva Hypothesis, the Earth is subject to increased asteroid impacts about once every 27 million years.',
        'Sustained and significant global cooling could kill many polar and temperate species and force others to migrate towards the equator.',
        'It has been suggested that global cooling caused or contributed to the End-Ordovician, Permian–Triassic, Late Devonian extinctions, and possibly others.',
        'Global warming as a cause of mass extinction is supported by several recent studies',
        'Their causes are complex and controversial, but all known instances are associated with severe and sustained global warming, mostly caused by sustained massive volcanism.',
        'It has been suggested that oceanic overturn caused or contributed to the late Devonian and Permian–Triassic extinctions.',
        'Humans, inturn, have greatly impacted marine species and ecosys-tems through the direct effects of exploitation and habitatdegradation, and the indirect effects of greenhouse gasemissions on ocean temperature, pH, and sea level',
        'Alvarez and his son, geologist Walter Alvarez, the hypothesis took time to gain acceptance, but buoyed by evidence like the Barringer Crater pictured below, an extraterrestrial impact is now the most widely accepted explanation for the Cretaceous-Paleogene (K-Pg) extinction.',
        'While the K-Pg extinction event was a very bad day for life on Earth, it was by no means the only one on record.',
        'What they have in common, though, is that they reshaped the face of life on Earth by wiping out a significant portion of it.',
        'Causes including volcanic eruptions, asteroid impacts, and a sudden release of greenhouse gasses from the seafloor have been proposed.',
        'Researchers have suggested climate change, an asteroid impact, or a spate of enormous volcanic eruptions as possible culprits—its effects are indisputable.',
        'The Holocene Extinction hasn’t been defined by a dramatic event like a meteor impact.',
        'Scientists from Stony Brook University and the Max Planck Institute of Animal Behavior have pieced together a timeline of how brain and body size evolved in mammals over the last 150 million years.',
        'The international team of 22 scientists, including biologists, evolutionary statisticians, and anthropologists, compared the brain mass of 1400 living and extinct mammals.',
        'The brain measurements were then analyzed along with body size to compare the scale of brain size to body size over deep evolutionary time.',
        'The study further showed that most changes in brain size occurred after two cataclysmic events in Earths history: the mass extinction 66 million years ago and a climatic transition 23-33 million years ago.',
        'After the mass extinction event at the end of the Cretaceous period, the researchers noticed a dramatic shift in brain-body scaling in lineages such as rodents, bats and carnivorans as animals radiated into the empty niches left by extinct dinosaurs.',
        'This includes evolution of the biggest mammalian brains, such as the dolphins, elephants, and great apes, which all evolved their extreme proportions after the climate change event 23-33 million years ago.',
        'The results suggest that as some tyrannosaurs grew older and heavier, their feet also became comparably more bulky, Mr Enriquez said.',
        'The answer lay in the Grande Prairie region of Northern Alberta, Canada, where the research team worked with well-preserved samples of footprints of different sizes that are suggested to belong to the same type of animal.',
        'The scientists found 70 of the 134 orders of ancient sea-dwelling invertebrates they examined could be identified as DCW in a new statistical analysis of the fossil record.',
        'We think of mass extinctions as being these selective forces that cause large groups of animals to go extinct, but our results really show there are a lot of instances where its not so sudden. It raises questions about why thats such a long delay.',
        'Paleontologist David Jablonski first coined the term DCW more than 20 years ago, and since then it has been associated almost exclusively with mass extinctions.',
        'The researchers concluded that the true extinction cascade was likely the result of complex, localised scenarios, including impacts of regional climate variation, and different pressures from people across regions.',
        'Our results support the notion that extinction risk can be high across all body sizes depending on a species particular ecology, meaning that predicting future extinctions from climate change and human impacts arent always straightforward based on the first principles of biology, concluded Professor Bradshaw.',
        'Earth-based catastrophes such as large-scale volcanism and global warming can destroy the ozone layer, too, but evidence for those is inconclusive for the time interval in question, Fields said.',
        'Tsunamis are giant waves caused by earthquakes or volcanic eruptions under the sea',
        'The understanding of tsunamis remained slim until the 20th century',
        'However, like tsunami, seismic sea wave is not a completely accurate term',
        'Examples of tsunamis originating at locations away'
        'Tsunami waves may travel as fast as jet planes over deep waters',
        'The team explored other astrophysical causes for ozone depletion, such as meteorite impacts, solar eruptions and gamma-ray bursts.',
        'When you see green bananas in Illinois, you know they are fresh, and you know they did not grow here. Like bananas, Pu-244 and Sm-146 decay over time. So if we find these radioisotopes on Earth today, we know they are fresh and not from here the green bananas of the isotope world and thus the smoking guns of a nearby supernova.',
        'Sedimentary mercury enrichments, proxies for massive volcanic events, have been detected in dozens of sedimentary rocks from the end of the Permian.',
        'We believe this to be the product of large volcanic eruptions because the coronene anomaly was formed by abnormally high temperature combustion, says professor Kaiho.',
        'Huge volcanic eruptions can produce sulfuric acid aerosols in the stratosphere and carbon dioxide in the atmosphere, which causes global climate changes.',
        'The coronene-mercury concentration firstly evidenced that volcanic hydrocarbon combustion helped contribute to the extinction through global warming.',
        'Professor Bradshaw and Dr Strona say that their virtual scenarios warn humanity not to underestimate the impact of co-extinctions.',
        'There is no convincing smoking gun, no compelling evidence of an asteroid impact.',
        'Another really important discovery was that in the case of global warming in particular, the combination of intolerance to heat combined with co-extinctions mean that 5-6 degrees of average warming globally is enough to wipe out most life on the planet, says Dr Strona.',
        'Waters and Carmichael will present the preliminary results of their research at the Geological Society of Americas Annual Meeting in Denver in October and at the American Geophysical Unions annual meeting in San Francisco in December.',
        'Wolves, dogs and dingoes are all species of the canidae family and are called canids',
        'Genetically and behaviorally they differ from dogs and are more like wolves in their inability to digest starches and their relationships with humans.',
        'With dingoes showing behaviors somewhere between wolves and dogs and exhibiting only slight genetic ability to consume starchy foods or tolerate captivity',
        'When a meteorite hurtles through the atmosphere and crashes to Earth, how does its violent impact alter the minerals found at the landing site?',
        'Sharks are predicted to be the most affected, with losses of functional richness far beyond those expected under random extinctions.',
        'Now, researchers including doctoral student Courtney Wagner and associate professor Peter Lippert from the University of Utah',
        'The Chicxulub asteroid impact that wiped out the dinosaurs likely released far more climate-altering sulfur gas into the atmosphere than originally thought, according to new research.',
        'Since 1996, Denver has worked in a number of paleontology museums and university departments, and also in science media; as a dig site leader for the BBCs Live from Dinosaur Island',
        'The Chicxulub impact occurred 66 million years ago when an asteroid approximately 12 kilometers (7 miles) wide slammed into Earth.',
        'Along with gaining a better understand of the Chicxulub impact, researchers can also use the new studys methods to estimate the amount of gas released during other large impacts in Earths history.',
        'It is a huge dinosaur, but we expect to find much more of the skeleton in future field trips, so we will have the possibility to address with confidence how really big it was, Alejandro Otero, a paleontologist with Argentinas Museo de La Plata, told CNN via email.',
        'A spokesman for the Myanmar government did not respond to CNN requests for comment.',
        'An emerging scientific consensus is that gases in particular carbon gases released by volcanic eruptions millions of years ago contributed to some of Earths greatest mass extinctions.',
        'Benjamin Black, assistant professor in CCNYs Division of Science, whose expertise includes effects of volcanism on climate and mass extinctions.'
        'It also helps us understand how volcanic eruptions move carbon',
        'This scale of CO2 release has only happened a few times in Earths history, for example during rare, enormous volcanic eruptions like the ones we studied.',
        'Consequently, Black pointed out, even though volcanic eruptions on the scale of these enormous volcanic provinces are not expected any time soon.',
        'The researchers used samples collected from ancient volcanic eruptions including the 252-million-year-old Siberian Traps.'
        'A study recently found that explosive volcanic eruptions were at least three to eight times more frequent during the peak of the Late Paleozoic Ice Age.'
        'Aerosols produced by explosive volcanism helped keep large ice sheets stable, even when CO2 levels increased, by blocking sunlight.',
        'During the LPIA, frequent explosive volcanism is thought to have caused increased reflection of sunlight, and increased atmospheric acidity.',
        'Explosive volcanism during this time interval peaked approximately 310-290 million years ago, right on time to keep climate cool and support CO2 uptake in the ocean.',
        'The iridium-laden dust is believed to be the shattered remains of the 200-km-wide asteroids impact.'
        'That observation is consistent with the expectation that particles were produced by the asteroid impact, since once the particles are ejected, heavier particles should fall back to Earth sooner (and travel shorter distances) than lighter particles.',
        'New evidence gleaned from Antarctic seashells confirms that Earth was already unstable before the asteroid impact that wiped out the dinosaurs.',
        'During the years leading up to the asteroid impact, the Deccan Traps spewed massive amounts of carbon dioxide (CO2) into the atmosphere.',
        'Our data suggest that the environment was changing before the asteroid impact.',
        'The asteroid impact coincides with pre-existing carbon cycle instability.',
        'Most of these surface rocks were basalts, and the asteroid impacts produced large pools of superheated impact melt of such composition.',
        'Asteroid impacts are supposed to play no role in the formation of the Earths early evolved crust.',
        'Professor Kirkland said the timing raised the possibility that the Earths oldest asteroid impact may have helped lift the planet out of a deep freeze.',
        'Associate Professor Nicholas Timms noted the precise coincidence between the Yarrabubba impact and the disappearance of glacial deposits.',
        'This twist of fate suggests that the large meteorite impact may have influenced global climate.',
        'Human activities could change the pace of evolution, similar to what occurred 66 million years ago when a giant asteroid wiped out the dinosaurs, leaving modern birds as their only descendants.',
        'A new reconstruction of Antarctic ocean temperatures around the time the dinosaurs disappeared 66 million years ago supports the idea that one of the planets biggest mass extinctions was due to the combined effects of volcanic eruptions and an asteroid impact.',
        'Many scientists believe the extinction was caused by an asteroid impact.'
       ]

nlp=spacy.load('en_core_web_sm')
ner=nlp.get_pipe("ner")

patterns = [
            [{"POS": "ADJ"}, {"LEMMA": "impact"}], #Impacts
            [{"POS": "NOUN"}, {"LEMMA": "impact"}], #Impacts
            [{"TEXT": "meteorite"}, {"LEMMA": "impact"}], #Impacts
    
            [{"POS": "NOUN"}, {"LEMMA": "strike"}], #Strikes
            [{"POS": "ADJ"}, {"LEMMA": "strike"}], #Strikes
    
            [{"POS": "NOUN"}, {"TEXT": "cooling"}], #Coling
            [{"POS": "ADJ"}, {"TEXT": "cooling"}], #Cooling
    
            [{"POS": "NOUN"}, {"TEXT": "warming"}], #Warming
            [{"POS": "ADJ"}, {"TEXT": "warming"}], #Warming
    
            [{"POS": "ADJ"}, {"TEXT": "volcanism"}], #Volcanism
            [{"POS": "ADJ"}, {"LEMMA": "eruption"}], #Volcanic eruption
            
            [{"TEXT": "oceanic"}, {"TEXT": "overturn"}], #Oceanic overturn
            [{"LEMMA": "tsunami"}]
           ]

matcher= Matcher(nlp.vocab)
matcher.add("DISASTER", patterns)

TRAINING_DATA= []
for doc in nlp.pipe(TEXTS):
    spans= [doc[start:end] for match_id, start, end in matcher(doc)]
    entities= [(span.start_char, span.end_char, "DISASTER") for span in spans]
    training_example= (doc.text, {"entities": entities})
    TRAINING_DATA.append(training_example)
    
print(TRAINING_DATA, sep="\n")

[('Scientists have long believed—at least before humanity became a force for extinction that there were just two ways to wipe out life on Earth: an asteroid strike or massive volcanic eruptions.', {'entities': [(145, 160, 'DISASTER'), (172, 190, 'DISASTER')]}), ('What the end-Devonian lacked was a cause. There was no evidence for volcanism or a giant impact, but one alluring clue was seen in the rapid formation.', {'entities': [(83, 95, 'DISASTER')]}), ('The Cretaceous-Paleogene extinction event is the most recent mass extinction and the only one definitively connected to a major asteroid impact.', {'entities': [(128, 143, 'DISASTER')]}), ('The massive impact which left a crater more than 120 miles wide—flung huge volumes of dust, debris, and sulfur into the atmosphere, bringing on severe global cooling.', {'entities': [(4, 18, 'DISASTER'), (151, 165, 'DISASTER')]}), ('According to the Shiva Hypothesis, the Earth is subject to increased asteroid impacts about once every 27 million year

In [11]:
#Add labels to the `ner`
for _, annotations in TRAINING_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

#Disable pipeline components
pipe_exceptions= ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes= [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
optimizer= nlp.resume_training()

with nlp.disable_pipes(*unaffected_pipes):
    #15 iterations
    for i in range(15):
        random.shuffle(TRAINING_DATA)

        for raw_data, entity_offsets in TRAINING_DATA:
            doc= nlp.make_doc(raw_data)
            example= Example.from_dict(doc, {"entities": entity_offsets['entities']})
            nlp.update([example], sgd=optimizer)

doc = nlp("Extinction is the death of all members of a species of plants, animals, or other organisms. One of the most dramatic examples of a modern extinction is the passenger pigeon.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []


In [30]:
"""
Catastrophes:
Asteroid impact/strike: 60
Global warming: 29
Volcanic eruption/activity: 29
Giant viruses: 6

Flood/Deluge: No matches
Oceanic reptiles, animals, fishes, conditions

"""

#Get all entities
ents= []
for doc in data:
    doco= nlp(doc['doc'])    
    for ent in doco.ents:
        ents.append((ent.text, ent.label_))

#Get DISASTER entities
specs= []
for enti in ents:
    if(enti[1] == 'DISASTER'):
            specs.append(enti[0])

scps= []
for scp in specs:
    if(len(scp) > 11):
        if(not ',' in scp):
            scps.append(scp)

#from nltk.probability import FreqDist
print(FreqDist(scps).most_common(20))

#Ocean mentions with DISASTER
for scp in specs:
    if('ocean' in scp):
        print(scp)

[('asteroid impact', 29), ('global warming', 29), ('volcanic eruptions', 25), ('asteroid strike', 18), ('volcanic activity', 16), ('volcanic ash', 13), ('global scale', 7), ('global distribution', 5), ('terrestrial vertebrates', 5), ('solar system', 4), ('global ecosystems', 4), ('nitrogen isotopes', 4), ('embryonic bones', 4), ('nitrogen isotope', 4), ('I. intermedius', 4), ('Smithsonian Voices', 3), ('Asteriornis maastrichtensis', 3), ('after impact', 3), ('ancient lake', 3), ('Ngwevu intloko', 3)]
oceanic reptiles
oceanic animals
global oceans
oceanic conditions
oceanic dolphins
oceanic region
oceanic fishes


# 2. What do they say about Million years?
### Spacy Matcher

In [34]:
"""
Spacy Matcher: Ages/Million years

"""

nlp= spacy.load('en_core_web_sm')
matcher= Matcher(nlp.vocab)
pattern= [{"IS_DIGIT": True}, {"TEXT": "million"}, {"TEXT": "years"}]

matcher.add("YEARS", [pattern])

all_origins= []
for doc in data:
    mat= nlp(doc['doc'])
    matches= matcher(mat)
    
    if matches:
        for match_id, start, end in matches:
            all_origins.append(mat[start:end].text)

FreqDist(all_origins).most_common(15)

[('66 million years', 129),
 ('100 million years', 95),
 ('65 million years', 51),
 ('200 million years', 50),
 ('150 million years', 44),
 ('20 million years', 38),
 ('10 million years', 38),
 ('50 million years', 36),
 ('70 million years', 36),
 ('160 million years', 34),
 ('250 million years', 30),
 ('75 million years', 30),
 ('120 million years', 29),
 ('80 million years', 29),
 ('5 million years', 29)]

In [102]:
"""
66 million years: Extinction of dinosaurs, asteroid
- A catastrophe that struck 66 million years ago.
- More than 75% of species known from the end of the cretaceous period didnt make it to the paleogene period.
- A catastrophic asteroid impact triggered a devastating mass extinction.
- An asteroid came screaming out of the sky and smacked into what is now the yucatán peninsula.

100 million years: Several species, discovers, ambar mentions, cretaceous
- New mantis and hairy cicada species, and legged snakes.
- Angiosperms only evolved and diversified about 100 million years ago.
- About 100 million years ago, the so called kem kem beds were home to a vast river system.
- Dubbed afrotapejara zouhrii, the newly discovered flying reptile.
- Australia was much closer to the south pole.

200 million years: Species appearances
- Pleurodires appear to have evolved.
- Metoposaurus algarvensis, an amphibian that roamed the earth. Basically a salamander thats the size of a car.
- Mammals and dinosaurs appeared at nearly the same time around.
- The origin of feathers was pushed back at least to the origin of those bird like dinosaurs.

"""

nlp= spacy.load('en_core_web_sm')
matcher= Matcher(nlp.vocab)
pattern= [{"TEXT": {"REGEX": "[A-Z][\w+]"}}, {"IS_PUNCT": False, "OP": "*"}, {"TEXT": "200"}, {"TEXT": "million"}, {"TEXT": "years"}, {"TEXT": {"REGEX": "[\w+]"}}, {"TEXT": "."}]
matcher.add("YEARS", [pattern])

all_origins= []
for doc in data:
    mat= nlp(doc['doc'])
    matches= matcher(mat)
    
    if matches:
        for match_id, start, end in matches:
            all_origins.append(mat[start:end].text)
            print(mat[start:end].text)

Earths orbital cyclesthe orientation of our planet on its axis and the shape of its path around the suninfluenced tropical climates and caused lakes to come and go as far back as 200 million years ago.
Olsen demonstrated that Earths orbital cyclesthe orientation of our planet on its axis and the shape of its path around the suninfluenced tropical climates and caused lakes to come and go as far back as 200 million years ago.
Mammals and dinosaurs appeared at nearly the same time around 200 million years ago.
Connecticut 200 million years ago.
Jurassic boundary 200 million years ago.
Triassic Jurassic boundary 200 million years ago.
Era 65 200 million years ago.
Mesozoic Era 65 200 million years ago.


# 3. Cambrian explosion/Sudden change mentions
### Nltk regex

In [3]:
"""
Nltk tokenize

"""

arts_content= []
for dat in data:
    #Nltk tokenize
    tokens= tokenize.word_tokenize(dat['doc'].lower())
    textf= nltk.Text(tokens)
    arts_content.append(textf)
    
print(len(arts_content))

1450


In [24]:
"""
Nltk regex: Cambrian explosion and trilobites
Cambrian explosion:
- Paleontologists are still discussing whether the evolutionary ----event was a real, 
sudden change, or if it simply looks that way because of incomplete fossil sampling.-----
- Unlike the single celled animals and microbes that lived before the geological age, 
these creatures had it all ----heads, tails, abdomens, shells, and lots of tiny little legs.----
- Researchers have long suspected that the cambrian explosion wasnt quite as explosive 
as it seems, and that in fact, ----some of these structures, especially appendages, may 
have evolved in the eons before the cambrian period.-----

Trilobites or complex fossils:
- Based on the fossil record, some of them have argued that contemporary animal 
groups exploded onto earth, very quickly morphing from ----single celled organisms 
to complex multicellular animals in a relatively short geological time span.----

"""

for art in arts_content:
    art.findall(r"<[\w\.,]+>{60} <arthropods?> <[\w\.,]+>{60}")
    #art.findall(r"<[\w\.,]+>{40} <arthropods?> <[\w\.,]+>{,20} <complex|complexity> <[\w\.,]+>{65}|<[\w\.,]+>{40} <complex|complexity> <[\w\.,]+>{,20} <arthropods?> <[\w\.,]+>{65}")






















. more than that , aphelicophontes danjuddi isnt merely a fragment or
an impression . the entire animal down to its reproductive anatomy has
been preserved . genitalia are important characteristics of insects
that are often used to describe and define species , smith says ,
especially because they are so seldom seen in fossil assassin bugs .
for these arthropods , genital anatomy is as distinctive as a
fingerprint in determining whos who . finding direct evidence of
fossil genitals is relatively rare . even in cases where mating
animals have been foundsuch as courting sharks or prehistoric turtles
caught en flagrantethe actual soft tissue anatomy is usually missing .
the same goes for insects . fossil arthropods have been





























































the years , several candidates have come forth , all known only by
their fossilized remains . two of the most promising possibilities
include many legged millipedes , eager to snack on t





















remains of a previously unknown species of eurypterid sea scorpion and
found direct evidence that these marine creatures were able to breathe
in subaerial environments through their main respiratory organs . the
new species , named adelophthalmus pyrrhae , lived about 350 million
years ago during the carboniferous period . it belongs to eurypterida
, a large group of extinct arthropods that thrived from the ordovician
through the permian period . their closest living relatives are
horseshoe crabs , which lay eggs on land but are unable to breathe
above water . the three dimensionally preserved specimen of
adelophthalmus pyrrhae was found 25 years ago in the lydiennes
formation in montagne noire region , france . we wondered if we could






































































































babcock , a paleontologist in the school of earth sciences at the ohio
state university . the inner workings of the respiratory circ






































today , the origin of mandibulates had largely remained a mystery ,
said dr. cédric aria , a postdoctoral researcher at the nanjing
institute for geology and palaeontology , china , and the lead author
of a report published this week on tokummia katalepsis in the journal
nature . before now weve had only sparse hints at what the first
arthropods with mandibles could have looked like , and no idea of what
could have been the other key characteristics that triggered the
unrivaled diversification of that group . tokummia katalepsis lived in
tropical seas during the cambrian period , approximately 508 million
years ago . this creature was a bottom dweller , as lobsters or mantis
shrimps today , and






in the welsh borderland is globally important as a source of
unparalleled paleobiological and phylogenetic data on a diversity of
mid silurian 430 million years ago invertebrate animals , prof.
siveter and colleagues said . these include a brachiopod , 



























































































preserved in three dimensions are quite rare and our x ray techniques
have allowed us to reveal this exceptional fossil in more detail than
we would have dreamed possible just a couple of decades ago , said dr
russell garwood from the university of manchester , the lead author of
the paper appearing in the journal current biology . terrestrial
arthropods like harvestmen have a sparse fossil record because their
exoskeletons dont preserve well . as a result , some fundamental
questions in the evolutionary history of these organisms remain
unresolved . this exceptional fossil has given us a rare and detailed
look at the anatomy of harvestmen that lived hundreds of millions of
years ago , explained study co author




















imaged sequential surfaces of the fossil , spaced a mere 20 microns
apart . these numerous images were then edited on a computer , in
places pixel by pixel , to identify

# 4. Carbon 14/Soft tissues found in fossils
### Nltk concordance

In [20]:
"""
Nltk concordance: Carbon 14 and soft tissues
Carbon 14: 
- No matches

Soft tissues:
- Finding preserved soft tissues is rare.
- The preservation of soft parts such as the antennae and legs in fossilized 
trilobites is extremely rare.
- Its very rare to see the preservation of soft tissues like organs or 
appendages in trilobites.
- Soft tissues like feathers and fur are rarely preserved in the fossil record.
- Mary Schweitzer and colleagues announced that they had found the remnants of 
blood vessels and other soft tissues in the femur of a tyrannosaurus.

- The combination of oil and salt prevented the remains from decomposing, 
allowing the soft tissues to remain intact.
- Highly acidic and low oxygen body of water shortly after its death. 
This allowed the soft tissues to become mineralized before they decayed away 
completely, so that they could be preserved.

"""

for art in arts_content:
    art.concordance("carbon", width=220)

no matches
no matches
Displaying 1 of 1 matches:
bined the standard physical study of the fossils with chemical analyses using rare earth elements , stable carbon and oxygen isotopes , and charcoal concentrations , reports juliet eilperin for the washington post . none
no matches
no matches
no matches
no matches
no matches
Displaying 6 of 6 matches:
 an ecosystem with an open canopy . for the second method , the researchers checked the ratio of a pair of carbon isotopescarbon 13 and carbon 12to infer how much sun beat down on a leaf when it was alive . if a collecti
nopy . for the second method , the researchers checked the ratio of a pair of carbon isotopescarbon 13 and carbon 12to infer how much sun beat down on a leaf when it was alive . if a collection of a forests leaves has ro
at down on a leaf when it was alive . if a collection of a forests leaves has roughly consistent ratios of carbon 12 to carbon 13 isotopes , then the forest probably had an open canopy . but if the forests 

no matches
no matches
Displaying 1 of 1 matches:
ent manner . global climates were much warmer 50 million years ago , associated with increased atmospheric carbon , a relationship that scientists see today . along with this , winters were mild , even in the cool , high
no matches
no matches
Displaying 1 of 1 matches:
 la laguna and dr josep antoni alcover from the mediterranean institute for advanced studies in mallorca . carbon dating has shown that the fossils are about 1,970 years old . the paleontologists have named the new speci
no matches
no matches
Displaying 1 of 1 matches:
e musser © australian museum scientists led by dr gilbert price from the university of queensland analyzed carbon isotope ratios present in fossil teeth of protemnodon sp. , macropus sp. , the extinct kangaroo troposodon
no matches
Displaying 9 of 9 matches:
biological in origin , they looked at 15 different samples of farrel quartzite and determined their stable carbon isotope ratios . the percentage of car

# 5. Catastrophes/Extinctions relations
### Nltk similar

In [14]:
"""
Ntlk similar: Extinctions
Flood/Deluge:
- No matches

Extinction:
- Asteroid, fossils, evolution, cataclysm, impacts, volcanism

Cataclysm:
- Impact, water, extinction, extinct

"""

for art in arts_content:
    art.similar("ocean")

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches

No matches
No matches
No matches
No matches

No matches

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches


No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches
No matches
No matches
No matches

No matches
No matches

No matches
study
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No ma

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches
No matches
No matches

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches
No matches

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches

No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
No matches
better
No matches

No matches
No matc

### Gensim most_simmilar

In [125]:
"""
Gensim most similar: Flood/Extinctions
Flood/Deluge:
- No matches

Extinction:
- period, end, triassic, cretaceous, million

Catastrophe:
- explosion, warm, strike, rainforest, cold

Global:
- warming, climatic, death, catastrophic, erosion

"""

#Split by sentences
dts= []
for da in data[:600]:
    dts.append(da['doc'].split('.')[:-1])    
dts = [item for sublist in dts for item in sublist]

#Remove stopwords
train_sents = [remove_stopwords(t) for t in dts]

#Lemmatize and tokenize
nlp= spacy.load('en_core_web_sm')
aer= []
for i in train_sents:
    do= nlp(i)
    sdf= " ".join([token.lemma_ for token in do])
    corpus= list(gensim.utils.tokenize(sdf.strip().lower()))
    aer.append(corpus)
#print(aer)

#Word2Vec
model= gensim.models.Word2Vec(aer, min_count= 5, size= 32)
vocab= list(model.wv.vocab)
print(len(vocab))

vectors= model.wv
vectors.save_word2vec_format("vect.txt", binary = False)

5235


In [168]:
model.wv.most_similar('death', topn=15)

[('geologic', 0.9984128475189209),
 ('indian', 0.9983055591583252),
 ('jehol', 0.9981716275215149),
 ('tropic', 0.9976853728294373),
 ('at', 0.9976083040237427),
 ('eruption', 0.9975685477256775),
 ('migration', 0.9973674416542053),
 ('cycle', 0.9973045587539673),
 ('route', 0.9971527457237244),
 ('global', 0.9971276521682739),
 ('greenland', 0.9970971345901489),
 ('olsen', 0.9970682859420776),
 ('warming', 0.997064471244812),
 ('biota', 0.9970595836639404),
 ('vast', 0.9970232248306274)]

# 6. Fossil record
### Spacy complex components

In [51]:
"""
Spacy complex components: Extinction causes

Pleistocene: Climate change
- The late pleistocene extinction, the event was likely caused by changes in climate.

Cretaceous: Asteroid
- About 65 million years ago the cretaceous era came to a dramatic end when a huge asteroid slammed into the Earth.
- Asteroid colliding with the Earth.

Triassic: Global warming
- Gases from enormous volcanic eruptions led to a spike in carbon dioxide levels, 
which in turn increased global temperatures by as much as 11 degrees F°.

Pliocene: Radiation
- Perhaps triggered by a supernova that sent harmful radiation toward Earth.

Flood causes:
- No matches

"""

nlp= spacy.load("en_core_web_sm")

ages= ['precambrian', 'cambrian', 'ordovician', 'silurian', 'devonian', 'carboniferous',
       'permian', 'triassic', 'jurassic', 'cretaceous', 'paleogene', 'neogene', 'quaternary',
       'holocene', 'pleistocene', 'pliocene', 'miogene', 'oligocene', 'eocene', 'paleocene']
ages_patterns= list(nlp.pipe(ages))
print("ages_patterns:", ages_patterns)

name= 'AGES'
matcher= PhraseMatcher(nlp.vocab)
matcher.add(name, ages_patterns)

@Language.component('ages')
def el_component(doc):
    matches= matcher(doc)
    spans= [Span(doc, start, end, label=name) for match_id, start, end in matches]
    doc.ents= spans
    
    return doc

nlp.add_pipe('ages', after="ner")
print()

all_sents= []
for doc in data[:400]:
    doco= nlp(doc['doc'].lower())
    
    ret= ""
    for ent in doco:
        if (ent.ent_type_ == 'AGES'):
            try:
                for j in reversed(range(60)):
                    ret= ret+" "+doco[ent.i-j].text

                for j in range(1, 60):
                    ret= ret+" "+doco[ent.i+j].text
                ret= ret+" -----"
            except:
                pass
    
    if 'extinction' in ret:
        #if any(word in ret for word in [' water ', ' storm ', ' flood ', ' deluge ', ' inundation ', ' rainforest ']):
        print(ret)
        print()

ages_patterns: [precambrian, cambrian, ordovician, silurian, devonian, carboniferous, permian, triassic, jurassic, cretaceous, paleogene, neogene, quaternary, holocene, pleistocene, pliocene, miogene, oligocene, eocene, paleocene]

 dinosaurs roamed the earth , flying reptiles known as pterosaurs ruled the skies . these leathery winged flyers reached enormous size , with some species unfurling wings that spanned 30 feet across or more . the first pterosaurs appeared around 220 million years ago and disappeared from the planet along with the dinosaurs 66 million years ago during the cretaceous <<<< paleogene mass extinction . for 200 years , researchers have been unable to trace the evolutionary path this group of reptiles took to get airborne . now , newly described fossils may clarify the evolutionary origins of the first vertebrates to develop powered flight , reports riley black for scientific american . the new research , published last week ----- roamed the earth , flying reptiles

 museum of natural history says most of the fossils you ll find at this site are from the late paleocenespecifically the aquia formation rock layer , which is 55 to 59 million years old . during this time period , many types of animals were evolving and diversifying to adapt to the extinction event at the end of the cretaceous <<<< time period , pinsdorf writes in an email to smithsonian magazine . because the deposit is marine in origin , the teeth of a wide variety of shark and bony fish species can be found in this formation . alongside shark teeth , you may find an occasional prehistoric crocodile tooth , which by comparison sort of look like ----- hillside covered in vegetation . its only a good spot for fossil collecting because it was mined for so long , he says . you re looking at bedrock where there should be soil . today , dinosaur park features fossilized dino bones , including some from marylands state dinosaur astrodon johnstoni , and floral remnants from the early cretace

 . as we continue to learn more about the diversity of these critters and their body plans , were slowly gathering the information that we need to better understand how they moved over 70 million years ago . habib says understanding these giant flying hunters is also crucial for getting a full understanding of how life evolved during the cretaceous <<<< . these are among the most popular and charismatic of all fossil animals , he says in the release . they have been inspiration for countless movie monsters , they were critical parts of global ecosystems worldwide during the age of dinosaurs , so they are key to understanding the ecology and extinctions of that time , and , -----

 of core samples taken from the craters inner ring of mountains , called a peak ring , geologists can create a detailed timeline of what happened on the day after impact . the immense chicxulub crater is a remnant of one of the most consequential days in the history of life on earth . the asteroid strike trigg

 , and who survived , set the stage for the next 66 million yearsincluding our own origin 300,000 years ago . the chicxulub impact was a catastrophic transition into a new world . the distinctive rock layer it left behind , spiked with an element called iridium often found in asteroids and meteorites , marks the end of the cretaceous <<<< period and the beginning of the paleogene , known by experts as the kpg boundary . this line in the stone is also the marker for the end of the age of dinosaurs and the beginning of the age of mammals , a shift that has been intensely debated and studied for decades . now a fossil site in ----- stage for the next 66 million yearsincluding our own origin 300,000 years ago . the chicxulub impact was a catastrophic transition into a new world . the distinctive rock layer it left behind , spiked with an element called iridium often found in asteroids and meteorites , marks the end of the cretaceous period and the beginning of the paleogene <<<< , known by

 of mexico . the devastation that followed was unprecedented , with tsunamis , an overheated atmosphere , darkened skies , a terrible cold snap , and other apocalyptic ecological events clearing away an estimated seventy five percent of known life on earth . paleontologists know this catastrophe as the kpg extinction event because it marks the transition from the cretaceous <<<< into the paleogene period of earths history . but even though it has been studied constantly , the details of this event still puzzle experts . the case was nt closed with the recognition of the impact crater in the 1990s , and exactly how the extinction played outwhat differentiated the living from the deadcontinues to inspire paleontologists to ----- the devastation that followed was unprecedented , with tsunamis , an overheated atmosphere , darkened skies , a terrible cold snap , and other apocalyptic ecological events clearing away an estimated seventy five percent of known life on earth . paleontologists k

 part of the body except the skull . the specimen is also larger than most of its relatives , about the size of a turkey vulture while others are closer to crow size or smaller . this particular species , dubbed mirarce eatoni , is about 75 million years old , meaning it dates from the end of the cretaceous <<<< period . it is described in a new paper published in the journal peerj . the completeness of the fossil allowed researchers to figure out how the birds flew and shows that by the end of the cretaceous some enantiornithines had developed many features that made them good flyers with traits similar to those that modern birds developed after ----- species , dubbed mirarce eatoni , is about 75 million years old , meaning it dates from the end of the cretaceous period . it is described in a new paper published in the journal peerj . the completeness of the fossil allowed researchers to figure out how the birds flew and shows that by the end of the cretaceous <<<< some enantiornithin

 what these kinds of studies show is that ecosystems , although remarkably resilient , really do have breaking points , jablonski says . and that history should be considered extremely sobering . sixty six million years ago , nothing seemed more unlikely than the dominance of bipedal apes and flying dinosaurs . yet here we are .   the cretaceous <<<< was a world of enormous terrestrial dinosaurs , some small mammals , and what we now recognize as the predecessors to modern birds . some , like hesperornis , were flightless creatures with a beak full of teeth that lived in the ocean . others , like icthyornis , were flying fish eaters . most diverse of all were ----- it is possible to study the composition of the flora and its change through time with very high precision , as you can sample the rock record centimeter by centimeter , said antoine bercovici , a paleobotanist at the smithsonian institution and an author on the new paper , by email . those microfossils from the boundary betw

 more archaic , categorized as an archosauriforman early member of the larger group that includes crocodiles , dinosaurs and their relatives .  
 triopticus    annotated partial skull 
     by witmerlab at ohio university 
     on sketchfab 
  this was an oddball among oddballs . none of the close relatives that we have for   triopticus   in the triassic <<<< have a similar structure to their heads , stocker says . nor is it clear why the animal evolved such a distinctive skull . its difficult for us to say what this domed morphology would have been for or what would have encouraged the evolution of this structure , she says . while paleontologists debate whether the copycat pachycephalosaurs ----- says . while paleontologists debate whether the copycat pachycephalosaurs used their skulls for display , combat or something else , why   triopticus   pioneered this look is a mystery . 

 nevertheless , the discovery of this dome headed animal highlights a curious pattern in the age of rep

 other mass extinction events   may deserve a closer look . just as the history books are written by the victors , the fossil record is dominated by survivors . but in both fields , recognizing the   bias helps   provide a clearer picture of the history of life on earth . about 65 million years ago the cretaceous <<<< era came to a dramatic end when a huge asteroid slammed into the earth and likely jump started the mass extinction that wiped out the dinosaurs . in the wake of such devastation , plucky mammals in their underground burrows survived and eventually rose to the prominence they enjoy today .     at least , that s the narrative ----- that specimen after specimen was yet another long extinct species that once crawled , skittered or scuttled across the surface of ancient earth . in short , there were a lot more dead ends on the tree of life than he expected . so longrich and his colleagues set out to quantify how the mammals fared during the end cretaceous <<<< extinction . the

 sandstone . the construction project was sidelined and further excavations at the site revealed a swath of 2,000 footprints . about 1,500 of the tracks were reburied so that they might be preserved , while 500 remain visible , protected by a geodesic dome built in 1977 . paleontologists surmise that 200 million years ago , in the early jurassic <<<< period , a dinosaur called dilophosaurus made the tracks , which measure 10 to 16 inches in length and are spaced 3.5 to 4.5 feet apart . when i was first there i was probably about 9 years old , says carrano . it was just very dramatic . the footprints are on the spot where the dinosaur stepped ----- quarry , in central utah , is basically a giant trap filled with dinosaur bones , says carrano     more than 12,000 bones , in fact . in the past century , researchers have found specimens of more than 70 dinosaurs , mostly the meat eating species allosaurus , at the quarry , making it the densest deposit of jurassic <<<< dinosaur bones in th

# 7. Rapid burial
### Nltk regex

In [12]:
"""
Nltk regex: Rapid burial

- These fossils are assumed to result from rapid burial, which
preserves individual positions during interactions.
- Rapid burial is usually required for fossil preservation.
- Preservation requires a specific kind of environment the ground 
needs to be sticky enough so that the footprint leaves a mark, 
davies says, but not so wet that it gets washed away. You need 
that balance in order to capture and preserve them.
- These animals were quickly covered by fine sediment while they 
were still alive or just after their death.
- Alive or dead, an organism needs to be rapidly buried. 
Sand, mud, ashalmost any sediment will do.

- Once the waters began to recede, objects sank according to 
their weight, with the heaviest ones ending up at the bottom.
- The team believes that the Cairo forest was wiped out by a 
flood due to the presence of many fish fossils that were also 
visible on the surface of the quarry.

"""

for art in arts_content:
    art.findall(r"<[\w\.,]+>{60} <fossilization> <[\w\.,]+>{60}")
    #art.findall(r"<[\w\.,]+>{40} <fossils?> <[\w\.,]+>{,20} <deluge> <[\w\.,]+>{65}|<[\w\.,]+>{40} <deluge> <[\w\.,]+>{,20} <fossils?> <[\w\.,]+>{65}")





















































































at the local university . incredibly curious about the world around
him , scheuchzer amassed a large collection of fossils , which led him
to ponder their origin . and therein lies the tale of how scheuchzer
came to insist that the fossil he had acquired must have been a human
witness to the flood . for scheuchzer , the biblical flood was the
most plausible mechanism for the formation of fossils . he set out to
gather scientific evidencea proposition that aroused consternation
among his fellow townspeople who regarded scripture as the literal
truth and in no need for verification . indeed , the zürich censors
denied permission to print one of scheuchzers books until he had
purged all references; 13 million years old . they formed in a lake
inside an extinct volcanic crater lined by forest . historians have
puzzled over how a well respected medical doctor could possibly have
mistaken the skeleton of a gi